In [ ]:
#Instalação dos pacotes
#**********************
#install.packages("twitteR")
#install.packages("data.table")
#install.packages("igraph")
#install.packages("gsubfn")
#install.packages("wordcloud")
#install.packages("NLP")

options(repr.plot.width=8, repr.plot.height=6)

In [ ]:
library('twitteR')
library(data.table)
library(igraph)
library(gsubfn)
library(wordcloud)
library(NLP)
library(tm)

In [ ]:
#Autenticando no Twitter
#***********************
# Gerar as API Keys pelo site
# https://developer.twitter.com/en/apps

options(httr_oauth_cache=T)

consumer_key = " "
consumer_secret = " "
access_token = " "
access_secret = " "

setup_twitter_oauth( consumer_key, consumer_secret, access_token, access_secret)

In [ ]:
#Coletando dados
#***************
usuario <- "DiegoNogare"
perfil <- getUser(usuario)
location(perfil)
description(perfil)

In [ ]:
seguidores <- perfil$getFollowers()
length(seguidores)
seguidores <- rbindlist(lapply(seguidores,as.data.frame))

In [ ]:
quemSigo <- perfil$getFriends()
length(quemSigo)
quemSigo <- rbindlist(lapply(quemSigo,as.data.frame))

In [ ]:
#Meus seguidores com mais seguidores
#***********************************
SeguidoresQueTemMaisSeguidores <- seguidores[ order(-seguidores$followersCount) ,]
top10Seguidores <- head(SeguidoresQueTemMaisSeguidores,10 )

top10Seguidores <- top10Seguidores [,c('name', 'followersCount', 'screenName', 'location', 'profileImageUrl')]

top10Seguidores

In [ ]:
#Quem eu sigo que tem mais seguidores
#************************************
SigoQueTemMaisSeguidores <- quemSigo[ order(-quemSigo$followersCount) ,]
top10quemSigo <- head(SigoQueTemMaisSeguidores,10 )

top10quemSigo <- top10quemSigo [,c('name', 'followersCount', 'screenName', 'location', 'profileImageUrl')]

top10quemSigo

In [ ]:
# Ultimos tweets de quem eu sigo
#***********************************
tweets <- as.data.frame(matrix(c(NA,NA), nrow=1,ncol=2))

ultimosTweets <- 1
dez <- length(top10quemSigo$screenName)
i <- 1
while (i <= dez)
{
  tweets <- rbind(tweets, as.data.frame( matrix(c(top10quemSigo[i]$screenName,
                                                  userTimeline(top10quemSigo[i]$screenName, n=ultimosTweets)[[1]]$text
  ) ,nrow=ultimosTweets, ncol=2, byrow=T) )
  )
  i <- i + 1
}
colnames(tweets) <- c("Perfil", "Post")
tweetsQuemSigo <- tweets[!is.na(tweets$Perfil),]

tweetsQuemSigo

In [ ]:
#Retornar os 10 Trending Topics de São Paulo
#*******************************************
localizacao <- availableTrendLocations()
woeid <- (localizacao[(localizacao$country == 'Brazil'
                       & localizacao$name == 'São Paulo'),])
                       #& localizacao$name == 'Brazil'),])
TrendTopics <- head(getTrends(woeid[3]),10)
TrendTopics$name


In [ ]:
termo <- as.character(TrendTopics$name[1])

In [ ]:
totalTweets = 5000
#termo <- "#CiteUmTristeFato"
desde <- "2020-07-01" #Ano-Mês-Dia

posts <- searchTwitter(termo, n=totalTweets,
                       since=desde, lang = 'pt') 

posts <- twListToDF(posts)



In [ ]:
totalTweets <- nrow(posts)

In [ ]:
CaracteresParaMudar <- list( 'S'='S', 's'='s', 'Z'='Z', 'z'='z', 'À'='A', 'Á'='A', 'Â'='A', 'Ã'='A', 'Ä'='A', 'Å'='A', 'Æ'='A', 'Ç'='C', 'È'='E', 'É'='E',
                             'Ê'='E', 'Ë'='E', 'Ì'='I', 'Í'='I', 'Î'='I', 'Ï'='I', 'Ñ'='N', 'Ò'='O', 'Ó'='O', 'Ô'='O', 'Õ'='O', 'Ö'='O', 'Ø'='O', 'Ù'='U',
                             'Ú'='U', 'Û'='U', 'Ü'='U', 'Ý'='Y', 'Þ'='B', 'ß'='Ss', 'à'='a', 'á'='a', 'â'='a', 'ã'='a', 'ä'='a', 'å'='a', 'æ'='a', 'ç'='c',
                             'è'='e', 'é'='e', 'ê'='e', 'ë'='e', 'ì'='i', 'í'='i', 'î'='i', 'ï'='i', 'ð'='o', 'ñ'='n', 'ò'='o', 'ó'='o', 'ô'='o', 'õ'='o',
                             'ö'='o', 'ø'='o', 'ù'='u', 'ú'='u', 'û'='u', 'ý'='y', 'ý'='y', 'þ'='b', 'ÿ'='y' )

In [ ]:
LimparTexto <- function(texto){
  texto <- tolower(texto)
  texto <- gsub("\\s?(f|ht)(tp)(s?)(://)([^\\.]*)[\\.|/](\\S*)", texto, replacement="")
  texto <- gsub("\n", texto, replacement=" ")
  texto <- gsub(pattern="[[:punct:]]", texto, replacement="")
  texto <- removeWords(texto, stopwords("portuguese"))
  texto <- gsubfn(paste(names(CaracteresParaMudar),collapse='|'), CaracteresParaMudar, texto)
  texto <- iconv(texto,from="UTF-8",to="ASCII//TRANSLIT")
  return(texto)
}

In [ ]:
postsLimpos <- as.vector( as.character( LimparTexto(posts$text) ) )

head(postsLimpos)

In [ ]:
#Criar nuvem de textos
#*********************
prepararNuvem <- function()
{
  docs <- Corpus(VectorSource(postsLimpos))
  dtm <- TermDocumentMatrix(docs)
  m <- as.matrix(dtm)
  v <- sort(rowSums(m),decreasing=TRUE)
  d <- data.frame(word = names(v),freq=v)
  return(d)
}

In [ ]:
d <- prepararNuvem()

wordcloud(words = d$word, freq = d$freq, min.freq = 1
          ,max.words=150
          ,random.order=FALSE
          ,colors=brewer.pal(8, "Dark2") )


In [ ]:
updateStatus( paste("Ao procurar", toupper(termo), "desde", desde, "foram encontradas", totalTweets, "postagens." ) )